In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium
from collections import defaultdict


url = "https://127.0.0.1:9200/vic-weather/_search"


headers = {
    'Content-Type': 'application/json'
}


data = {
    "size": 100,
    "query": {
        "match_all": {}
    },
    "sort": [
        {
            "timestamp": {
                "order": "desc"
            }
        }
    ]
}


response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)

response_json = response.json()

hits = response_json['hits']['hits']

latest_records = {}
for hit in hits:
    source = hit['_source']
    coords = (source['geo']['lat'], source['geo']['lon'])
    if coords not in latest_records:
        latest_records[coords] = source


features = []
for coords, source in latest_records.items():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [coords[1], coords[0]]
        },
        "properties": {
            "stationid": source['stationid'],
            "timestamp": source['timestamp'],
            "name": source['name'],
            "air_temp": source['air_temp'],
            "apparent_t": source['apparent_t'],
            "rel_hum": source['rel_hum'],
            "rain_trace": source['rain_trace'],
            "wind_dir": source['wind_dir'],
            "wind_spd_kmh": source['wind_spd_kmh']
        }
    }
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

m = folium.Map(location=[-37.8, 145.0], zoom_start=8)

for feature in geojson_data['features']:
    lon, lat = feature['geometry']['coordinates']
    popup_content = f"""
    <b>Station:</b> {feature['properties']['name']}<br>
    <b>Temperature:</b> {feature['properties']['air_temp']} °C<br>
    <b>Apparent Temperature:</b> {feature['properties']['apparent_t']} °C<br>
    <b>Humidity:</b> {feature['properties']['rel_hum']}%<br>
    <b>Wind Direction:</b> {feature['properties']['wind_dir']}<br>
    <b>Wind Speed:</b> {feature['properties']['wind_spd_kmh']} km/h<br>
    <b>Timestamp:</b> {feature['properties']['timestamp']}
    """
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)

folium.LayerControl().add_to(m)

m.save('weather_map.html')

# uncomment the following line if you are running this script in Jupyter Notebook
m


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium

# Elasticsearch URL
bicycle_url = "https://127.0.0.1:9200/bicycle_count_2015-/_search"


headers = {
    'Content-Type': 'application/json'
}


data = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
    # ,
    # "sort": [
    #     {
    #         "begin_time": {
    #             "order": "desc"
    #         }
    #     }
    # ]
}


bicycle_response = requests.get(bicycle_url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
bicycle_data = bicycle_response.json()

# print the json response with formatted
print(json.dumps(bicycle_data, indent=2))


bicycle_hits = bicycle_data['hits']['hits']
bicycle_marker_data = []
for hit in bicycle_hits:
    source = hit['_source']
    lat = source['latitude']
    lon = source['longitude']
    count = source['count']
    description = source['descripti2']
    popup_content = f"""
    <b>Count:</b> {count}<br>
    <b>Description:</b> {description}<br>
    <b>Coordinates:</b> ({lat}, {lon})
    """
    bicycle_marker_data.append({
        'lat': lat,
        'lon': lon,
        'popup_content': popup_content
    })


m = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)

for data in bicycle_marker_data:
    folium.Marker(
        location=[data['lat'], data['lon']],
        popup=folium.Popup(data['popup_content'], max_width=300)
    ).add_to(m)

folium.LayerControl().add_to(m)

m.save('bicycle_data_map.html')

# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium
from folium.plugins import HeatMap
import numpy as np

url = "https://127.0.0.1:9200/air-qualities/_search"

headers = {
    'Content-Type': 'application/json'
}

data = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}

response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)

response_json = response.json()

hits = response_json['hits']['hits']

print(json.dumps(hits, indent=2))

heat_data = []
for hit in hits:
    source = hit['_source']
    lat = source['geometry']['lat']
    lon = source['geometry']['lon']
    value = source['averageValue']
    timestamp = source['since']
    
    if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(value, (int, float)):
        heat_data.append([lat, lon, value])

heat_data = [point for point in heat_data if not any(isinstance(i, (type(None), float)) and (not isinstance(i, bool)) and np.isnan(i) for i in point)]

m = folium.Map(location=[-37.8136, 144.9631], zoom_start=6)

HeatMap(heat_data, min_opacity=0.5, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(m)

geojson_path = 'states.geojson'
folium.GeoJson(
    geojson_path,
    name='Australian States',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

folium.LayerControl().add_to(m)

m.save('air_quality_heatmap.html')

# uncomment the following line to display the map
m


In [ ]:
import requests
import json
import pandas as pd
from dateutil import parser
from pytz import UTC
from requests.auth import HTTPBasicAuth
import folium
import math

air_quality_url = "https://127.0.0.1:9200/air-qualities/_search"
weather_url = "https://127.0.0.1:9200/vic-weather/_search"

headers = {
    'Content-Type': 'application/json'
}

query = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}

response_aq = requests.get(air_quality_url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=query, verify=False)
air_quality_data = response_aq.json()

response_weather = requests.get(weather_url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=query, verify=False)
weather_data = response_weather.json()

air_quality_records = []
for hit in air_quality_data['hits']['hits']:
    source = hit['_source']
    record = {
        'siteID': source['siteID'],
        'siteName': source['siteName'],
        'lat': float(source['geometry']['lat']),
        'lon': float(source['geometry']['lon']),
        'dataName': source['dataName'],
        'averageValue': source['averageValue'],
        'unit': source['unit'],
        'timestamp': parser.parse(source['since']).astimezone(UTC)
    }
    air_quality_records.append(record)

weather_records = []
for hit in weather_data['hits']['hits']:
    source = hit['_source']
    record = {
        'stationid': source['stationid'],
        'name': source['name'],
        'lat': float(source['geo']['lat']),
        'lon': float(source['geo']['lon']),
        'air_temp': source['air_temp'],
        'apparent_t': source['apparent_t'],
        'rel_hum': source['rel_hum'],
        'rain_trace': source['rain_trace'],
        'wind_dir': source['wind_dir'],
        'wind_spd_kmh': source['wind_spd_kmh'],
        'timestamp': parser.parse(source['timestamp']).astimezone(UTC)
    }
    weather_records.append(record)

df_air_quality = pd.DataFrame(air_quality_records)
df_weather = pd.DataFrame(weather_records)

print(df_air_quality.head())
print("\n\n\n")
print(df_weather.head())

df_combined = pd.merge_asof(
    df_air_quality.sort_values('timestamp'), 
    df_weather.sort_values('timestamp'), 
    on='timestamp', 
    by=['lat', 'lon'], 
    direction='nearest', 
    tolerance=pd.Timedelta('1h')
)

df_combined.dropna(inplace=True)

print(df_combined.head())

m = folium.Map(location=[-37.8136, 144.9631], zoom_start=8)

def calculate_arrow_end(lat, lon, wind_dir, wind_spd_kmh):
    arrow_length = wind_spd_kmh / 100.0

    angle_dict = {
        'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 'E': 90, 'ESE': 112.5,
        'SE': 135, 'SSE': 157.5, 'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
        'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5
    }
    angle = angle_dict.get(wind_dir, 0)

    end_lat = lat + arrow_length * math.cos(math.radians(angle))
    end_lon = lon + arrow_length * math.sin(math.radians(angle))

    return end_lat, end_lon

for index, row in df_weather.iterrows():
    lat, lon = row['lat'], row['lon']
    wind_dir = row['wind_dir']
    wind_spd_kmh = row['wind_spd_kmh']
    end_lat, end_lon = calculate_arrow_end(lat, lon, wind_dir, wind_spd_kmh)

    folium.PolyLine(
        locations=[(lat, lon), (end_lat, end_lon)],
        color='blue',
        weight=2,
        opacity=0.6,
        tooltip=f"Wind: {wind_dir} at {wind_spd_kmh} km/h"
    ).add_to(m)


    popup_content = f"""
    <b>Station:</b> {row['name']}<br>
    <b>Temperature:</b> {row['air_temp']} °C<br>
    <b>Apparent Temperature:</b> {row['apparent_t']} °C<br>
    <b>Humidity:</b> {row['rel_hum']}%<br>
    <b>Rain:</b> {row['rain_trace']} mm<br>
    <b>Wind:</b> {wind_dir} at {wind_spd_kmh} km/h
    """
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)


m.save('weather_wind_map.html')


# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium
from folium.plugins import HeatMap
import numpy as np


def get_twitter_heat_data():
    url = "https://127.0.0.1:9200/geo_twitter_data/_search"
    headers = {'Content-Type': 'application/json'}
    data = {
        "size": 1000,
        "query": {"match_all": {}},
        "sort": [{"created_at": {"order": "desc"}}]
    }
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    response_json = response.json()
    hits = response_json['hits']['hits']
    
    heat_data = []
    for hit in hits:
        source = hit['_source']
        bbox = source['bbox']['coordinates'][0]
        lon = sum([point[0] for point in bbox]) / len(bbox)
        lat = sum([point[1] for point in bbox]) / len(bbox)
        heat_data.append([lat, lon, source['sentiment']])
    
    return heat_data


def get_air_quality_heat_data():
    url = "https://127.0.0.1:9200/air-qualities/_search"
    headers = {'Content-Type': 'application/json'}
    data = {"size": 1000, "query": {"match_all": {}}}
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    response_json = response.json()
    hits = response_json['hits']['hits']
    
    heat_data = []
    for hit in hits:
        source = hit['_source']
        lat = source['geometry']['lat']
        lon = source['geometry']['lon']
        value = source['averageValue']
        
        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(value, (int, float)):
            heat_data.append([lat, lon, value])
    
    heat_data = [point for point in heat_data if not any(isinstance(i, (type(None), float)) and (not isinstance(i, bool)) and np.isnan(i) for i in point)]
    return heat_data


def get_vic_weather_heat_data():
    url = "https://127.0.0.1:9200/vic-weather/_search"
    headers = {'Content-Type': 'application/json'}
    data = {"size": 1000, "query": {"match_all": {}}}
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    response_json = response.json()
    hits = response_json['hits']['hits']
    
    heat_data = []
    for hit in hits:
        source = hit['_source']
        lat = source['geo']['lat']
        lon = source['geo']['lon']
        value = source['air_temp']
        
        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(value, (int, float)):
            heat_data.append([lat, lon, value])
    
    heat_data = [point for point in heat_data if not any(isinstance(i, (type(None), float)) and (not isinstance(i, bool)) and np.isnan(i) for i in point)]
    return heat_data


def get_nsw_weather_heat_data():
    url = "https://127.0.0.1:9200/nsw-weather/_search"
    headers = {'Content-Type': 'application/json'}
    data = {"size": 1000, "query": {"match_all": {}}}
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    response_json = response.json()
    hits = response_json['hits']['hits']
    
    heat_data = []
    for hit in hits:
        source = hit['_source']
        lat = source['geo']['lat']
        lon = source['geo']['lon']
        value = source['air_temp']
        
        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(value, (int, float)):
            heat_data.append([lat, lon, value])
    
    heat_data = [point for point in heat_data if not any(isinstance(i, (type(None), float)) and (not isinstance(i, bool)) and np.isnan(i) for i in point)]
    return heat_data


twitter_heat_data = get_twitter_heat_data()
air_quality_heat_data = get_air_quality_heat_data()
vic_weather_heat_data = get_vic_weather_heat_data()
nsw_weather_heat_data = get_nsw_weather_heat_data()


m = folium.Map(location=[-33.8688, 151.2093], zoom_start=4)


twitter_heatmap = folium.FeatureGroup(name='Twitter Sentiment Heatmap')
HeatMap(twitter_heat_data, min_opacity=0.5, radius=15, blur=10).add_to(twitter_heatmap)
twitter_heatmap.add_to(m)


air_quality_heatmap = folium.FeatureGroup(name='Air Quality Heatmap')
HeatMap(air_quality_heat_data, min_opacity=0.5, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(air_quality_heatmap)
air_quality_heatmap.add_to(m)


vic_weather_heatmap = folium.FeatureGroup(name='VIC Weather Heatmap')
HeatMap(vic_weather_heat_data, min_opacity=0.5, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(vic_weather_heatmap)
vic_weather_heatmap.add_to(m)


nsw_weather_heatmap = folium.FeatureGroup(name='NSW Weather Heatmap')
HeatMap(nsw_weather_heat_data, min_opacity=0.5, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(nsw_weather_heatmap)
nsw_weather_heatmap.add_to(m)


geojson_path = 'states.geojson'
state_boundaries = folium.FeatureGroup(name='Australian States')
folium.GeoJson(
    geojson_path,
    name='Australian States',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(state_boundaries)
state_boundaries.add_to(m)


folium.LayerControl().add_to(m)


m.save('combined_heatmap.html')


# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import folium
from folium.plugins import HeatMap
import numpy as np


url = "https://127.0.0.1:9200/vic-weather/_search"


headers = {
    'Content-Type': 'application/json'
}


data = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}


response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)


response_json = response.json()


hits = response_json['hits']['hits']


heat_data = []
for hit in hits:
    source = hit['_source']
    lat = source['geo']['lat']
    lon = source['geo']['lon']
    value = source['air_temp']
    

    if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(value, (int, float)):

        heat_data.append([lat, lon, value])


heat_data = [point for point in heat_data if not any(isinstance(i, (type(None), float)) and (not isinstance(i, bool)) and np.isnan(i) for i in point)]



m = folium.Map(location=[-37.8136, 144.9631], zoom_start=6)


HeatMap(heat_data, min_opacity=0.5, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'}).add_to(m)


geojson_path = 'states.geojson'
folium.GeoJson(
    geojson_path,
    name='Australian States',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)


folium.LayerControl().add_to(m)


m.save('vic_weather_heatmap.html')

# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import pandas as pd
from dateutil import parser
import pytz
import folium
from folium.plugins import HeatMapWithTime

url = "https://127.0.0.1:9200/vic-weather/_search"

headers = {
    'Content-Type': 'application/json'
}

data = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}

response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)

response_json = response.json()

hits = response_json['hits']['hits']

weather_records = []
for hit in hits:
    source = hit['_source']
    record = {
        'lat': source['geo']['lat'],
        'lon': source['geo']['lon'],
        'air_temp': source['air_temp'],
        'timestamp': parser.parse(source['timestamp']).astimezone(pytz.UTC)
    }
    weather_records.append(record)

df_weather = pd.DataFrame(weather_records)

print(df_weather.head())

heat_data = []
timestamps = []

for timestamp, group in df_weather.groupby(pd.Grouper(key='timestamp', freq='1H')):
    heat_data.append([[row['lat'], row['lon'], row['air_temp']] for _, row in group.iterrows()])
    timestamps.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

print(heat_data[:2])
print(timestamps[:2])

m = folium.Map(location=[-37.8136, 144.9631], zoom_start=6)

heatmap = HeatMapWithTime(
    heat_data,
    index=timestamps,
    radius=25,
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1: 'red'},
    auto_play=True,
    max_opacity=0.8
)

heatmap.add_to(m)

geojson_path = 'states.geojson'
folium.GeoJson(
    geojson_path,
    name='Australian States',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

folium.LayerControl().add_to(m)

m.save('vic_weather_timeslider_heatmap.html')

# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import requests
import pandas as pd
import json
from shapely.geometry import shape
import geopandas as gpd

url = 'http://localhost:9090/traffic-injury'

headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
}

response = requests.get(url, headers=headers)

print(f"HTTP response status code: {response.status_code}")

if response.status_code == 200:
    try:
        data = response.text

        print(f"HTTP response content: {data[:500]}...")

        data = json.loads(data)

        print(json.dumps(data, indent=2))

        if 'error' in data:
            print(f"Error from server: {data['error']}")
        else:
            suic_df = pd.DataFrame(data['data'])

            print(suic_df.columns)

            suic_df['geometry'] = suic_df['geometry'].apply(lambda x: shape(x) if x else None)

            gdf = gpd.GeoDataFrame(suic_df, geometry='geometry')

            geojson_file = 'traffic-injury.geojson'
            gdf.to_file(geojson_file, driver='GeoJSON')

            print(f"GeoJSON file saved as {geojson_file}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Response content: {data[:500]}...")
else:
    print(f"HTTP request failed with status code {response.status_code}")
    print(f"Error message: {response.text}")


In [ ]:
import folium
import geopandas as gpd


geojson_file = 'traffic-injury.geojson'
gdf = gpd.read_file(geojson_file)

m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=6)

folium.GeoJson(
    gdf,
    name='Traffic Injuries',
    style_function=lambda feature: {
        'fillColor': '#blue',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['phn_name', 'count_traffic_injury', 'ratio_traffic_injury'],
        aliases=['Region:', 'Traffic Injuries:', 'Injury Ratio:'],
        localize=True
    ),
    popup=folium.GeoJsonPopup(
        fields=['phn_name', 'count_traffic_injury', 'ratio_traffic_injury'],
        aliases=['Region:', 'Traffic Injuries:', 'Injury Ratio:'],
        localize=True
    )
).add_to(m)

folium.LayerControl().add_to(m)

m.save('traffic_injury_map.html')

m

In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

def get_vic_weather_data():
    url = "https://127.0.0.1:9200/vic-weather/_search"
    headers = {'Content-Type': 'application/json'}
    data = {
        "size": 1000,
        "query": {"match_all": {}},
        "sort": [{"timestamp": {"order": "desc"}}]
    }
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []

    response_json = response.json()
    hits = response_json.get('hits', {}).get('hits', [])
    
    weather_data = []
    for hit in hits:
        source = hit.get('_source', {})
        lat = source.get('geo', {}).get('lat')
        lon = source.get('geo', {}).get('lon')
        rain = source.get('rain_trace')
        if rain == '-':
            rain = 0.0
        else:
            try:
                rain = float(rain)
            except ValueError:
                rain = 0.0

        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(rain, (int, float)):
            weather_data.append([lat, lon, rain])
    
    print("VIC Weather Data:", weather_data)
    return weather_data

def get_nsw_weather_data():
    url = "https://127.0.0.1:9200/nsw-weather/_search"
    headers = {'Content-Type': 'application/json'}
    data = {
        "size": 1000,
        "query": {"match_all": {}},
        "sort": [{"timestamp": {"order": "desc"}}]
    }
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []

    response_json = response.json()
    hits = response_json.get('hits', {}).get('hits', [])
    
    weather_data = []
    for hit in hits:
        source = hit.get('_source', {})
        lat = source.get('geo', {}).get('lat')
        lon = source.get('geo', {}).get('lon')
        rain = source.get('rain_trace')
        if rain == '-':
            rain = 0.0
        else:
            try:
                rain = float(rain)
            except ValueError:
                rain = 0.0

        if isinstance(lat, (int, float)) and isinstance(lon, (int, float)) and isinstance(rain, (int, float)):
            weather_data.append([lat, lon, rain])
    
    print("NSW Weather Data:", weather_data)
    return weather_data

vic_weather_data = get_vic_weather_data()
nsw_weather_data = get_nsw_weather_data()

if not vic_weather_data:
    print("No VIC weather data found.")
if not nsw_weather_data:
    print("No NSW weather data found.")

traffic_injury_gdf = gpd.read_file('traffic-injury.geojson')

print("Traffic Injury GDF bounds:")
print(traffic_injury_gdf.total_bounds)

def check_overlap(bounds1, bounds2):
    return not (bounds1[0] > bounds2[2] or bounds1[2] < bounds2[0] or bounds1[1] > bounds2[3] or bounds1[3] < bounds2[1])

if vic_weather_data:
    vic_bounds = [min([p[1] for p in vic_weather_data]), min([p[0] for p in vic_weather_data]), max([p[1] for p in vic_weather_data]), max([p[0] for p in vic_weather_data])]
    print("VIC Weather data overlap:", check_overlap(traffic_injury_gdf.total_bounds, vic_bounds))

if nsw_weather_data:
    nsw_bounds = [min([p[1] for p in nsw_weather_data]), min([p[0] for p in nsw_weather_data]), max([p[1] for p in nsw_weather_data]), max([p[0] for p in nsw_weather_data])]
    print("NSW Weather data overlap:", check_overlap(traffic_injury_gdf.total_bounds, nsw_bounds))

if vic_weather_data:
    vic_weather_gdf = gpd.GeoDataFrame(vic_weather_data, columns=['lat', 'lon', 'rain'], geometry=gpd.points_from_xy([p[1] for p in vic_weather_data], [p[0] for p in vic_weather_data]), crs="EPSG:4326")

if nsw_weather_data:
    nsw_weather_gdf = gpd.GeoDataFrame(nsw_weather_data, columns=['lat', 'lon', 'rain'], geometry=gpd.points_from_xy([p[1] for p in nsw_weather_data], [p[0] for p in nsw_weather_data]), crs="EPSG:4326")

if vic_weather_data and vic_weather_gdf.crs != traffic_injury_gdf.crs:
    vic_weather_gdf = vic_weather_gdf.to_crs(traffic_injury_gdf.crs)
if nsw_weather_data and nsw_weather_gdf.crs != traffic_injury_gdf.crs:
    nsw_weather_gdf = nsw_weather_gdf.to_crs(traffic_injury_gdf.crs)

if vic_weather_data:
    vic_weather_gdf['coords'] = vic_weather_gdf['geometry'].apply(lambda geom: (geom.x, geom.y))
    vic_weather_grouped = vic_weather_gdf.groupby('coords').agg({
        'rain': 'mean',
        'geometry': 'first'
    }).reset_index()

if nsw_weather_data:
    nsw_weather_gdf['coords'] = nsw_weather_gdf['geometry'].apply(lambda geom: (geom.x, geom.y))
    nsw_weather_grouped = nsw_weather_gdf.groupby('coords').agg({
        'rain': 'mean',
        'geometry': 'first'
    }).reset_index()

print("Traffic Injury GDF Geometry Type:")
print(traffic_injury_gdf.geom_type.unique())

if vic_weather_data:
    print("VIC Weather GDF Sample:")
    print(vic_weather_gdf.head())
if nsw_weather_data:
    print("NSW Weather GDF Sample:")
    print(nsw_weather_gdf.head())
print("Traffic Injury GDF Sample:")
print(traffic_injury_gdf.head())

if vic_weather_data:
    vic_joined_gdf = gpd.sjoin(vic_weather_gdf, traffic_injury_gdf, how="inner", op='within')
if nsw_weather_data:
    nsw_joined_gdf = gpd.sjoin(nsw_weather_gdf, traffic_injury_gdf, how="inner", op='within')

if vic_weather_data:
    print("VIC Joined Data:")
    print(vic_joined_gdf.head())
if nsw_weather_data:
    print("NSW Joined Data:")
    print(nsw_joined_gdf.head())

if vic_weather_data and 'injury_ratio' in vic_joined_gdf.columns and 'rain' in vic_joined_gdf.columns:
    vic_grouped = vic_joined_gdf.groupby('phn_name').agg({
        'rain': 'mean',
        'injury_ratio': 'mean'
    }).reset_index()

    print("VIC Grouped Data:")
    print(vic_grouped)

    traffic_injury_gdf = traffic_injury_gdf.merge(vic_grouped, on='phn_name', suffixes=('_orig', '_vic'))
    print("Traffic Injury Data with VIC Grouped Values:")
    print(traffic_injury_gdf.head())

if nsw_weather_data and 'injury_ratio' in nsw_joined_gdf.columns and 'rain' in nsw_joined_gdf.columns:
    nsw_grouped = nsw_joined_gdf.groupby('phn_name').agg({
        'rain': 'mean',
        'injury_ratio': 'mean'
    }).reset_index()

    print("NSW Grouped Data:")
    print(nsw_grouped)

    traffic_injury_gdf = traffic_injury_gdf.merge(nsw_grouped, on='phn_name', suffixes=('_vic', '_nsw'))
    print("Traffic Injury Data with NSW Grouped Values:")
    print(traffic_injury_gdf.head())

m = folium.Map(location=[traffic_injury_gdf.geometry.centroid.y.mean(), traffic_injury_gdf.geometry.centroid.x.mean()], zoom_start=6)

folium.Choropleth(
    geo_data=traffic_injury_gdf,
    name='choropleth',
    data=traffic_injury_gdf,
    columns=['phn_name', 'ratio_traffic_injury'],
    key_on='feature.properties.phn_name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Traffic Injury (ratio_traffic_injury)'
).add_to(m)

# Custom MarkerCluster to display aggregated rainfall values
class CustomMarkerCluster(MarkerCluster):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def add_child(self, child, name=None, index=None):
        super().add_child(child, name, index)
        # Update cluster popups with rainfall sums
        for cluster in self._children.values():
            if isinstance(cluster, folium.plugins.marker_cluster.MarkerCluster):
                cluster_sum = 0
                for marker in cluster._children.values():
                    if isinstance(marker, folium.map.Marker):
                        try:
                            rain_value = float(marker.popup._content.split(': ')[1])
                            cluster_sum += rain_value
                        except:
                            continue
                cluster.bindPopup(f'Rainfall Sum: {cluster_sum:.2f}')

if vic_weather_data:
    marker_cluster = CustomMarkerCluster(name='VIC Rainfall Cluster', disableClusteringAtZoom=8).add_to(m)

    for idx, row in vic_weather_grouped.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=folium.Popup(f'Rainfall: {row["rain"]:.2f}'),
            icon=folium.DivIcon(html=f"""<div style="font-size: 12pt; color : black">{row["rain"]:.2f}</div>""")
        ).add_to(marker_cluster)

if nsw_weather_data:
    marker_cluster = CustomMarkerCluster(name='NSW Rainfall Cluster', disableClusteringAtZoom=8).add_to(m)

    for idx, row in nsw_weather_grouped.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=folium.Popup(f'Rainfall: {row["rain"]:.2f}'),
            icon=folium.DivIcon(html=f"""<div style="font-size: 12pt; color : black">{row["rain"]:.2f}</div>""")
        ).add_to(marker_cluster)

geojson_path = 'states.geojson'
state_boundaries = folium.FeatureGroup(name='Australian States')
folium.GeoJson(
    geojson_path,
    name='Australian States',
    style_function=lambda feature: {
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(state_boundaries)
state_boundaries.add_to(m)

folium.LayerControl().add_to(m)

m.save('traffic_injury_weather_map.html')
print("Map has been saved as 'traffic_injury_weather_map.html'")

# uncomment the following line to display the map in Jupyter Notebook
m


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

vic_weather_df = pd.DataFrame(vic_weather_data, columns=['lat', 'lon', 'rain'])
nsw_weather_df = pd.DataFrame(nsw_weather_data, columns=['lat', 'lon', 'rain'])

vic_weather_df['state'] = 'VIC'
nsw_weather_df['state'] = 'NSW'

combined_weather_df = pd.concat([vic_weather_df, nsw_weather_df])

plt.figure(figsize=(10, 6))
sns.boxplot(x='state', y='rain', data=combined_weather_df)
plt.title('Box Plot of Rainfall in VIC and NSW')
plt.xlabel('State')
plt.ylabel('Rainfall (mm)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

vic_weather_df = pd.DataFrame(vic_weather_data, columns=['lat', 'lon', 'rain'])
nsw_weather_df = pd.DataFrame(nsw_weather_data, columns=['lat', 'lon', 'rain'])

vic_weather_df['state'] = 'VIC'
nsw_weather_df['state'] = 'NSW'

combined_weather_df = pd.concat([vic_weather_df, nsw_weather_df])

plt.figure(figsize=(10, 6))
sns.kdeplot(data=combined_weather_df, x='rain', hue='state', fill=True, common_norm=False, alpha=0.5)
plt.title('Density Plot of Rainfall in VIC and NSW')
plt.xlabel('Rainfall (mm)')
plt.ylabel('Density')
plt.show()


In [ ]:
print(traffic_injury_gdf.describe())
if vic_weather_data:
    print(vic_weather_gdf.describe())
if nsw_weather_data:
    print(nsw_weather_gdf.describe())


In [ ]:

if vic_weather_data:
    print("VIC Joined Data Columns:")
    print(vic_joined_gdf.columns)

if nsw_weather_data:
    print("NSW Joined Data Columns:")
    print(nsw_joined_gdf.columns)


In [ ]:
import requests
import json
import pandas as pd
from dateutil import parser
from pytz import UTC
from requests.auth import HTTPBasicAuth

air_quality_url = "https://127.0.0.1:9200/air-qualities/_search"
weather_url = "https://127.0.0.1:9200/nsw-weather/_search"

headers = {
    'Content-Type': 'application/json'
}

query = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}

response_aq = requests.get(air_quality_url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=query, verify=False)
air_quality_data = response_aq.json()

response_weather = requests.get(weather_url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=query, verify=False)
weather_data = response_weather.json()

air_quality_records = []
for hit in air_quality_data['hits']['hits']:
    source = hit['_source']
    record = {
        'siteID': source['siteID'],
        'siteName': source['siteName'],
        'lat': float(source['geometry']['lat']),
        'lon': float(source['geometry']['lon']),
        'dataName': source['dataName'],
        'averageValue': source['averageValue'],
        'unit': source['unit'],
        'timestamp': parser.parse(source['since']).astimezone(UTC)
    }
    air_quality_records.append(record)

weather_records = []
for hit in weather_data['hits']['hits']:
    source = hit['_source']
    record = {
        'stationid': source['stationid'],
        'name': source['name'],
        'lat': float(source['geo']['lat']),
        'lon': float(source['geo']['lon']),
        'air_temp': source['air_temp'],
        'apparent_t': source['apparent_t'],
        'rel_hum': source['rel_hum'],
        'rain_trace': source['rain_trace'],
        'wind_dir': source['wind_dir'],
        'wind_spd_kmh': source['wind_spd_kmh'],
        'timestamp': parser.parse(source['timestamp']).astimezone(UTC)
    }
    weather_records.append(record)


df_air_quality = pd.DataFrame(air_quality_records)
df_weather = pd.DataFrame(weather_records)


print(df_air_quality.head(10))
print("\n\n\n")
print(df_weather.head(10))


df_combined = pd.merge_asof(
    df_air_quality.sort_values('timestamp'), 
    df_weather.sort_values('timestamp'), 
    on='timestamp', 
    by=['lat', 'lon'], 
    direction='nearest', 
    tolerance=pd.Timedelta('1h')
)


df_combined.dropna(inplace=True)


print(df_combined.head())


In [ ]:
import requests
import json
import pandas as pd
from dateutil import parser
from requests.auth import HTTPBasicAuth


air_quality_url = "https://127.0.0.1:9200/air-qualities/_search"
vic_weather_url = "https://127.0.0.1:9200/vic-weather/_search"
nsw_weather_url = "https://127.0.0.1:9200/nsw-weather/_search"


headers = {
    'Content-Type': 'application/json'
}


query = {
    "size": 1000,
    "query": {
        "match_all": {}
    }
}


def get_data(url):
    response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=query, verify=False)
    return response.json()


air_quality_data = get_data(air_quality_url)
vic_weather_data = get_data(vic_weather_url)
nsw_weather_data = get_data(nsw_weather_url)


def extract_data(data, lat_key, lon_key, value_key, timestamp_key):
    records = []
    for hit in data['hits']['hits']:
        source = hit['_source']
        record = {
            'lat': float(source['geo'][lat_key]),
            'lon': float(source['geo'][lon_key]),
            'value': source[value_key],
            'timestamp': parser.parse(source[timestamp_key])
        }
        records.append(record)
    return pd.DataFrame(records)

def extract_aq_data(data, lat_key, lon_key, value_key, timestamp_key):
    records = []
    for hit in data['hits']['hits']:
        source = hit['_source']
        record = {
            'lat': float(source['geometry'][lat_key]),
            'lon': float(source['geometry'][lon_key]),
            'value': source[value_key],
            'timestamp': parser.parse(source[timestamp_key])
        }
        records.append(record)
    return pd.DataFrame(records)


df_air_quality = extract_aq_data(air_quality_data, 'lat', 'lon', 'averageValue', 'since')
df_vic_weather = extract_data(vic_weather_data, 'lat', 'lon', 'air_temp', 'timestamp')
df_nsw_weather = extract_data(nsw_weather_data, 'lat', 'lon', 'air_temp', 'timestamp')


print(df_air_quality.head())
print(df_vic_weather.head())
print(df_nsw_weather.head())

# sort the dataframes by values
df_air_quality = df_air_quality.sort_values('value')
df_vic_weather = df_vic_weather.sort_values('value')
df_nsw_weather = df_nsw_weather.sort_values('value')

# print the sorted dataframes
print(df_air_quality.head())
print(df_vic_weather.head())
print(df_nsw_weather.head())


In [ ]:
import branca.colormap as cm
from folium.plugins import TimeSliderChoropleth

def prepare_timeslider_data(df, value_name):

    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['lon'], row['lat']],
            },
            "properties": {
                "time": row['timestamp'].strftime("%Y-%m-%dT%H:%M:%S"),
                value_name: row['value']
            },
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features,
    }


timeslider_data_aq = prepare_timeslider_data(df_air_quality, 'averageValue')
timeslider_data_vic_weather = prepare_timeslider_data(df_vic_weather, 'air_temp')
timeslider_data_nsw_weather = prepare_timeslider_data(df_nsw_weather, 'air_temp')


all_timeslider_data = {
    "type": "FeatureCollection",
    "features": timeslider_data_aq['features'] + timeslider_data_vic_weather['features'] + timeslider_data_nsw_weather['features']
}

# print all_timeslider_data with format
print(json.dumps(all_timeslider_data, indent=4))


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


url = "https://127.0.0.1:9200/history_weather/_search"


headers = {
    'Content-Type': 'application/json'
}


data = {
    "size": 1000,
    "query": {
        "bool": {
            "should": [
                {"range": {"Date": {"gte": "2023-04-01", "lte": "2023-04-30"}}},
                {"range": {"Date": {"gte": "2024-04-01", "lte": "2024-04-30"}}}
            ]
        }
    }
}


response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)


response_json = response.json()


with open('weather_data.json', 'w', encoding='utf-8') as f:
    json.dump(response_json, f, ensure_ascii=False, indent=4)


with open('weather_data.json', 'r', encoding='utf-8') as f:
    response_json = json.load(f)


hits = response_json['hits']['hits']


weather_data = []
for hit in hits:
    source = hit['_source']
    weather_data.append({
        "Date": source["Date"],
        "MinTemp(°C)": source["MinTemp(°C)"],
        "MaxTemp(°C)": source["MaxTemp(°C)"]
    })


df = pd.DataFrame(weather_data)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# print DataFrame head
print(df.head())


df_2023 = df.loc['2023-04']
df_2024 = df.loc['2024-04']


plt.figure(figsize=(14, 7))


plt.plot(df_2023.index, df_2023['MinTemp(°C)'], label='2023.4 Min. Temp(°C)', marker='o')
plt.plot(df_2023.index, df_2023['MaxTemp(°C)'], label='2023.4 Max. Temp(°C)', marker='x')


plt.plot(df_2024.index, df_2024['MinTemp(°C)'], label='2024.4 Min. Temp(°C)', marker='o')
plt.plot(df_2024.index, df_2024['MaxTemp(°C)'], label='2024.4 Max. Temp(°C)', marker='x')


plt.title('2023.4 and 2024.4 daily Min. and Max. Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(True)


plt.show()


In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


url = "https://127.0.0.1:9200/history_weather/_search"


headers = {
    'Content-Type': 'application/json'
}


data = {
    "size": 1000,
    "query": {
        "bool": {
            "should": [
                {"range": {"Date": {"gte": "2023-04-01", "lte": "2023-04-30"}}},
                {"range": {"Date": {"gte": "2024-04-01", "lte": "2024-04-30"}}}
            ]
        }
    }
}


response = requests.get(url, headers=headers, auth=HTTPBasicAuth('elastic', 'elastic'), json=data, verify=False)


response_json = response.json()


with open('weather_data.json', 'w', encoding='utf-8') as f:
    json.dump(response_json, f, ensure_ascii=False, indent=4)


with open('weather_data.json', 'r', encoding='utf-8') as f:
    response_json = json.load(f)


hits = response_json['hits']['hits']


weather_data = []
for hit in hits:
    source = hit['_source']
    weather_data.append({
        "Date": source["Date"],
        "MinTemp(°C)": source["MinTemp(°C)"],
        "MaxTemp(°C)": source["MaxTemp(°C)"]
    })


df = pd.DataFrame(weather_data)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# print DataFrame head
print(df.head())


df_2023 = df.loc['2023-04']
df_2024 = df.loc['2024-04']


df_2023['AvgTemp(°C)'] = (df_2023['MinTemp(°C)'] + df_2023['MaxTemp(°C)']) / 2
df_2024['AvgTemp(°C)'] = (df_2024['MinTemp(°C)'] + df_2024['MaxTemp(°C)']) / 2


df_2023['Day'] = df_2023.index.day
df_2024['Day'] = df_2024.index.day


plt.figure(figsize=(14, 7))


plt.plot(df_2023['Day'], df_2023['AvgTemp(°C)'], label='2023.4 Avg. Temp.(°C)', marker='o')


plt.plot(df_2024['Day'], df_2024['AvgTemp(°C)'], label='2024.4 Avg. Temp.(°C)', marker='x')


plt.title('2023.4 and 2024.4 daily Avg. Temperatures')
plt.xlabel('Date')
plt.ylabel('Temperatures (°C)')
plt.legend()
plt.grid(True)


plt.show()
